In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torch import tensor
from torch.autograd import Variable
from generate_data import generate_data, data_loader

In [13]:
batch_size = 1

lr = 0.0001

loss_fn = nn.CrossEntropyLoss()

In [14]:
N = 4*int(1e4)

X, y = generate_data(N, is_short=True, type = 2)

X_test, y_test = generate_data(2000, is_short=True, type = 2)

In [15]:
class RNN_predictor(nn.Module):
    def __init__(self, input_dim, hidden_dim, layer_dim, output_dim):
        super(RNN_predictor, self).__init__()
        
        # Number of hidden dimensions
        self.hidden_dim = hidden_dim
        
        # Number of hidden layers
        self.layer_dim = layer_dim
        
        # RNN
        self.rnn = nn.RNN(input_dim, hidden_dim, layer_dim, nonlinearity='relu')
        
        # Readout layer
        self.fc = nn.Linear(hidden_dim, output_dim)
    
    def forward(self, x):
        
        # Initialize hidden state with zeros
        h0 = Variable(torch.zeros(self.layer_dim, x.size(0), self.hidden_dim))
            
        # One time step
        out, hn = self.rnn(x, h0)
        out = self.fc(out[:, -1, :]) 
        return out


model = RNN_predictor(3, 100, 1, 2)

In [16]:
print(model)

RNN_predictor(
  (rnn): RNN(3, 100)
  (fc): Linear(in_features=100, out_features=2, bias=True)
)


In [17]:
def test(model):
    model.eval()
    test_batches = data_loader(X_test,y_test,batch_size=batch_size, min_length = 20)
    acc = 0.0
    i = 0
    for data, labels in test_batches:
        pred = model(data).view((-1,2))
        corr = (torch.argmax(pred,1) == torch.argmax(labels,1)).float().mean().item()
        acc += corr
        i += 1
    return acc/i

def train(model,n_epochs):
    parameters = model.parameters()

    optimizer = torch.optim.Adam(parameters, lr=lr)

    model.train()


    for i in range(n_epochs):
        model.train()
        train_batches = data_loader(X,y,batch_size=batch_size, min_length = 20)

        sum_loss = 0.0
        i_loss = 0
        for data, labels in train_batches:
            pred = model(data)
            loss_fn = F.binary_cross_entropy
            loss = loss_fn(pred,labels)
            loss.backward()
            optimizer.step()

            sum_loss += loss.item()
            i_loss += 1
        print("loss",sum_loss/i_loss)
        acc = test(model)
        print("acc",acc)


train(model,10)

RuntimeError: Expected hidden size (1, 1, 100), got (1, 20, 100)

In [21]:
train_batches = data_loader(X,y,batch_size=batch_size, min_length = 20)

t = next(train_batches)

In [22]:
t[0].shape

torch.Size([50, 32, 3])